In [88]:
import os
from time import sleep
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait

# import chromedriver_autoinstaller
# chromedriver_autoinstaller.install()

In [89]:
opt = webdriver.ChromeOptions()
opt.add_argument("--start-maximized")
opt.add_experimental_option("excludeSwitches", ["disable-popup-blocking"])
opt.add_argument("--headless")
# opt.add_argument("--lang=en")

In [90]:
WRITE_FLAG=True

In [91]:
root_data_folder="../data/raw/campsandclinincs/"
output_folder_name="mass_elite"
output_file_name=pd.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
output_file_path = os.path.join(root_data_folder,output_folder_name)
if (not os.path.exists(output_file_path)):
    os.makedirs(output_file_path)


<ipython-input-91-f0dfd2c40d5b>:3: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  output_file_name=pd.datetime.now().strftime("%Y-%m-%d %H:%M:%S")


In [92]:
base_link="https://www.masselite.com/team/college-camps-and-clinics/"

In [93]:
def load_webpage_dynamically(
        weblink:str,
        driver_obj:object,
        sleep_time=2,
        increment=1,
        limit=10
    ):
    """to load webpage recursively"""
    print(f'sleep time: {sleep_time} increment: {increment}')
    print(f"getting {weblink}")
    driver_obj.get(weblink)
    if limit < sleep_time+increment:
        return False
    sleep(sleep_time)
    # check if page is loaded
    if not EC.presence_of_element_located('.//table[@class="tablepress tablepress-id-47 dataTable no-footer"]'):
        return load_webpage_dynamically(weblink=weblink,driver_obj=driver_obj,sleep_time=sleep_time+increment,increment=increment+1,limit=limit)
    return True

In [94]:
def fetch_table_data():
    """ fetchs table data from loaded page """
    table_element = WebDriverWait(driver, 20).until(
                EC.presence_of_element_located((By.XPATH, './/table[@class="tablepress tablepress-id-47 dataTable no-footer"]'))
            )
    df_main =  pd.read_html(table_element.get_attribute("outerHTML"))[0]
    all_links=[]
    rows = table_element.find_elements(by=By.XPATH, value=".//tr")
    # Iterate through each row
    for row in rows:
        # Extract the cells of the current row
        cells = row.find_elements(by=By.XPATH, value=".//td")
        # Check if the row has cells
        if cells:
            # Get the last cell of the row
            last_cell = cells[-1]
            # Check if the last cell contains a link
            link = last_cell.find_element(by=By.XPATH, value=".//a[@href]").get_attribute("href")
            all_links.append(link)
    df_main['CAMP LINK']=all_links
    return df_main

In [95]:
def main():
    """ main execution of scraping bot """
    driver = webdriver.Chrome(options=opt)
    try:
        if load_webpage_dynamically(base_link,driver):
            print("featching table data now ... ")
            output_df=fetch_table_data()
            if WRITE_FLAG:
                output_df.to_csv(f"{output_file_path}/{output_file_name}.csv")
                print(f'completed,\n{output_df.shape} record found, \n {output_file_name} File saved successfully...')

    except Exception as e:
        print(f"could fetch data, exception accured")
        print(e)
    driver.close()

In [96]:
main()

sleep time: 2 increment: 1
getting https://www.masselite.com/team/college-camps-and-clinics/
featching table data now ... 
completed,
(62, 5) record found, 
 2024-04-03 17:19:38 File saved successfully...
